<a href="https://colab.research.google.com/github/shahriarabdullah/CNN_Microstructure/blob/main/GoogleColab/CNN_Microstructure_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convolutional Neural Network to Predict Grain Class (Coarse/Fine)

Abdullah Shahriar<br>
31.01.22

**NOTE:** Run each of the following cells serially to train and predict result from the model.

In [ ]:
# Download dataset from my GitHub
!wget https://github.com/shahriarabdullah/CNN_Microstructure/raw/main/GoogleColab/test.zip
!wget https://github.com/shahriarabdullah/CNN_Microstructure/raw/main/GoogleColab/train.zip


In [ ]:
# Extract from ZIP
!unzip "train.zip" -d "/content/"
!unzip "test.zip" -d "/content/"

In [ ]:
# Download samples for prediction
!wget https://github.com/shahriarabdullah/CNN_Microstructure/raw/main/GoogleColab/c1.png
!wget https://github.com/shahriarabdullah/CNN_Microstructure/raw/main/GoogleColab/c2.png
!wget https://github.com/shahriarabdullah/CNN_Microstructure/raw/main/GoogleColab/f1.png
!wget https://github.com/shahriarabdullah/CNN_Microstructure/raw/main/GoogleColab/f2.png

In [ ]:
# Train model

import tensorflow as tf
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import pandas as pd

tf.random.set_seed(45)

train = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)
test = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)

train_dataset = train.flow_from_directory("/content/train/",
                                          target_size=(432,288),
                                          batch_size = 2,
                                          class_mode = 'binary')
                                         
test_dataset = test.flow_from_directory("/content/test/",
                                          target_size=(432,288),
                                          batch_size =2,
                                          class_mode = 'binary')

test_dataset.class_indices

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(8,(3,3),activation='relu',input_shape=(432,288,3)),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Conv2D(16,(3,3),activation='relu'),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Conv2D(32,(3,3),activation='relu'),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Conv2D(32,(3,3),activation='relu'),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(1,activation='sigmoid')

])	

model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"]
)


history=model.fit_generator(train_dataset,
         steps_per_epoch = 12,
         epochs = 10,
         validation_data = test_dataset
         )

model.save("trained_model.h5")

pd.DataFrame(history.history).plot()

In [ ]:
# Function for prediction
def predict_ms(filename):
    img = tf.keras.preprocessing.image.load_img(filename,target_size=(432,288))
    
    plt.figure(figsize=(3,7))
    plt.imshow(img)
    
    Y = tf.keras.preprocessing.image.img_to_array(img)
    
    X = np.expand_dims(Y,axis=0)
    val = model.predict(X)
    print(val)
    if val == 1:
        
        plt.xlabel("FINE MICROSTRUCTURE",fontsize=20)
        
    
    elif val == 0:
        
        plt.xlabel("COARSE MICROSTRUCTURE",fontsize=20)

In [ ]:
# Run the following code to predict result

# predict_ms("c1.png") # Output should be COARSE MICROSTRUCTURE

predict_ms("f1.png") # Ouput should be FINE MICROSTRUCTURE